In [17]:
import pandas  as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


train=pd.read_csv("C:\\Users\\Prerit\\Desktop\\projects\\Artificial Intelligence- Deep Learning\\NLP\\NLP disaster tweets\\data\\train.csv")
test=pd.read_csv("C:\\Users\\Prerit\\Desktop\\projects\\Artificial Intelligence- Deep Learning\\NLP\\NLP disaster tweets\\data\\test.csv")

In [14]:
train.head

<bound method NDFrame.head of          id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...  

In [15]:
rows,columns=map(int,train.shape)
print("Number of rows =" ,rows,"\nNumber of columns =",columns,"\n")

print(train.isnull().any(),"\n\n")

null_columns=train.columns[train.isnull().any()] #Stores all columns which have null types in them
#print(null_columns)

print(train[null_columns].isnull().sum(),"\n\n")

Number of rows = 7613 
Number of columns = 5 

id          False
keyword      True
location     True
text        False
target      False
dtype: bool 


keyword       61
location    2533
dtype: int64 




In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Prerit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prerit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Prerit\AppData\Roaming\nltk_data...


True

In [5]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

In [ ]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)# Fit to the data and transform to feature matrix
train = vectoriser.fit_transform(train['text'])# Convert sparse matrix to dataframe
train = pd.DataFrame.sparse.from_spmatrix(train)# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}# Rename each column using the mapping
for col in train.columns:
    train.rename(columns={col: col_map[col]}, inplace=True)
train

In [19]:
count_vectorizer = feature_extraction.text.CountVectorizer(analyzer=preprocess_text)

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train["text"][0:5])